<a href="https://colab.research.google.com/github/ssawant/llm-forge/blob/main/foundational_models/pytorch_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer
`model.py`

In [6]:
import torch
import torch.nn as nn
import math

### Input Embedding

In [5]:
class InputEmbeddings(nn.Module):

    def __init__(self,d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
      return self.embedding(x) * math.sqrt(self.d_model)

### Positional Encoding

In [7]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)

        # Create a matrix os shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        # Create a vector of shape (seq_len, 1)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # Apply the sin to even position
        pe[:, 0::2] = torch.sin(position * div_term)

        # Apply the cos to odd position
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0) # (1, seq_len, d_model)

        # Now save the position encoding
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.size[1], :]).requires_grad_(False) # pos enco will not change during training
        return self.dropout(x)



## Encoder

### Layer normalization

In [ ]:
class LayerNormalization(nn.Module):

      def __init__(self, eps: float = 10**6) -> None:
          super().__init__()
          self.eps = eps
          self.alpha = nn.Parameter(torch.ones(1)) # Multiplied
          self.bias = nn.Parameter(torch.zeros(1)) # Added

      def forward(self, x):
          mean = x.mean(dim=-1, keepdim=True)
          std = x.std(dim=-1, keepdim=True)
          return self.alpha * (x - mean) / (std + self.eps) + self.bias

### Feed Forward

In [ ]:
class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # W1 and B1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) # W2 and B2

    def forward(self, x):
      # (Batch, Seq_Len, d_model) --> (Batch, seq_len, d_ff) --> (Batch, Seq_Len, d_model)
      return self.linear_2(self.dropout(self.linear_1(x)))